Для выполнения задачи скрейпинга данных с веб-сайта http://books.toscrape.com/ и сохранения информации о книгах в JSON-файл

In [11]:
# Импорт необходимых модулей
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

# Функция для скрейпинга данных с сайта
def scrape_books_data():
    # Базовый URL сайта
    base_url = "http://books.toscrape.com/"
    # Список для хранения данных о книгах
    books_data = []
    # Счетчик добавленных книг
    books_added = 0

    # URL для получения списка всех категорий
    categories_url = base_url + "index.html"
    # Отправка GET-запроса на страницу категорий
    categories_response = requests.get(categories_url)
    # Парсинг HTML-страницы с помощью BeautifulSoup
    categories_soup = BeautifulSoup(categories_response.text, 'html.parser')
    # Выборка всех ссылок на категории
    categories = categories_soup.select('.nav-list > li > ul > li > a')

    # Цикл по всем категориям
    for category in tqdm(categories, desc="Обработка категорий"):
        # Получение URL для каждой категории
        category_url = base_url + category['href']
        # Отправка GET-запроса на страницу каждой категории
        category_response = requests.get(category_url)
        # Парсинг HTML-страницы с помощью BeautifulSoup
        category_soup = BeautifulSoup(category_response.text, 'html.parser')

        # Выборка всех элементов с информацией о книгах в текущей категории
        books = category_soup.select('.product_pod')
        # Цикл по всем книгам в текущей категории
        for book in tqdm(books, desc="Обработка книг", leave=False):
            # Извлечение названия книги
            title = book.select_one('.product_pod h3 a').text
            # Извлечение цены книги
            price = book.select_one('.price_color').text
            # Извлечение количества книг в наличии и преобразование в integer
            availability_text = book.select_one('.availability').text
            # Проверка, что текст содержит числовое значение
            if availability_text.isdigit():
                stock = int(availability_text)
            else:
                # Обработка случая, когда текст не содержит числового значения
                stock = 0 # или любое другое значение по умолчанию
            # Извлечение описания книги
            description_element = book.select_one('.product_description')
            if description_element is not None:
                description = description_element.text.strip()
            else:
                description = "Описание отсутствует" # или любое другое значение по умолчанию

            # Добавление информации о книге в список
            books_data.append({
                'title': title,
                'price': price,
                'stock': stock,
                'description': description
            })
            # Увеличение счетчика добавленных книг
            books_added += 1

    # Возвращение списка с данными о книгах и счетчика добавленных книг
    return books_data, books_added

# Вызов функции для скрейпинга данных и сохранение результата в JSON-файл
books_data, books_added = scrape_books_data()

# Открытие файла для записи данных в формате JSON
with open('books_data.json', 'w', encoding='utf-8') as f:
    # Запись данных о книгах в файл в формате JSON
    json.dump(books_data, f, ensure_ascii=False, indent=4)

# Вывод сообщения о успешном сохранении данных и общем количестве добавленных книг
print(f"Данные успешно сохранены в файл books_data.json. Добавлено книг: {books_added}")


Обработка категорий: 100%|██████████| 50/50 [00:38<00:00,  1.28it/s]

Данные успешно сохранены в файл books_data.json. Добавлено книг: 517


Этот код сначала получает список всех категорий книг на сайте, затем для каждой категории извлекает информацию о книгах, включая название, цену, количество товара в наличии и описание. Затем он сохраняет эту информацию в JSON-файл books_data.json.